In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
IMG_SIZE = (224,224)  
BATCH_SIZE = 32
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(),'sample'))
TRAIN_DIR = os.path.join(BASE_DIR,'train')
VALID_DIR = os.path.join(BASE_DIR,'valid')

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 40,
    width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2,
    zoom_range = 0.2, horizontal_flip = True, fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size = IMG_SIZE,
    batch_size = BATCH_SIZE, class_mode='binary')
valid_datagen = ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_directory(VALID_DIR, target_size = IMG_SIZE,
    batch_size = BATCH_SIZE, class_mode='binary')

Found 17502 images belonging to 2 classes.
Found 7498 images belonging to 2 classes.


In [3]:
base_model = VGG16(weights='imagenet', include_top = False, input_shape = (224,224,3))
base_model.trainable = False
model = models.Sequential([base_model, layers.Flatten(), layers.Dense(256, activation='relu'),
    layers.Dropout(0.5), layers.Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer = optimizers.Adam(learning_rate = 1e-4),
    metrics = ['accuracy'])

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only = True,         
    save_weights_only = False, mode='max', verbose = 1)
history = model.fit(train_generator, steps_per_epoch = train_generator.samples, epochs = 10,
    validation_data = valid_generator, validation_steps = valid_generator.samples,
    callbacks = [checkpoint])

c:\Users\hankc\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
  547/17502 ━━━━━━━━━━━━━━━━━━━━ 23:35:46 5s/step - accuracy: 0.7727 - loss: 0.4676

c:\Users\hankc\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 1: val_accuracy improved from -inf to 0.90771, saving model to best_model.h5


17502/17502 ━━━━━━━━━━━━━━━━━━━━ 3817s 218ms/step - accuracy: 0.8218 - loss: 0.3886 - val_accuracy: 0.9077 - val_loss: 0.2270
Epoch 2/10
  284/17502 ━━━━━━━━━━━━━━━━━━━━ 21:43:23 5s/step - accuracy: 0.8612 - loss: 0.3147

KeyboardInterrupt: 